In [1]:
import re
import sys
import numpy as np
import pandas as pd
from pymagnitude import *
import matplotlib.pyplot as plt
%matplotlib inline
import gc
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import RNN, GRU, LSTM, Dense, Input, Embedding, Dropout, Activation, concatenate
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import initializers, regularizers, constraints, optimizers, layers
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D, SimpleRNN
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import pandas as pd
import os
import nltk
import re
from bs4 import BeautifulSoup
import urllib3
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools
from sklearn import preprocessing
from scipy import sparse
from keras import backend as K # Importing Keras backend (by default it is Tensorflow)
from keras.layers import Input, Dense # Layers to be used for building our model
from keras.models import Model # The class used to create a model
from keras.optimizers import Adam
from keras.utils import np_utils # Utilities to manipulate numpy arrays
from tensorflow import set_random_seed # Used for reproducible experiments
from tensorflow import keras
import gc
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import InputLayer, Input, Embedding, Dense, Dropout, Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, SpatialDropout1D, Conv1D, CuDNNLSTM, CuDNNGRU, TimeDistributed, Reshape, Permute, LocallyConnected1D, concatenate, ELU, Activation, add, Lambda, BatchNormalization, PReLU, MaxPooling1D, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras import regularizers
#from kgutil.models.keras.base import DefaultTrainSequence, DefaultTestSequence
#from kgutil.models.keras.rnn import KerasRNN, load_emb_matrix
from copy import deepcopy
import inspect

import os

/opt/conda/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/allennlp/commands/find_learning_rate.py:55: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')
Using TensorFlow backend.


### Read/Transformat data
- Read dataset
- Split comments(x) and categories(y)
- Tokenize all the comment (take the max_features most frequent words of all the comments)
- Pad each comment to max_len

In [2]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')

In [3]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_data[classes].values

train_sentences = train_data["comment_text"].fillna("fillna").str.lower()
test_sentences = test_data["comment_text"].fillna("fillna").str.lower()

max_features = 150000
max_len = 150
embed_size = 300

tokenizer = Tokenizer(max_features)
tokenizer.fit_on_texts(list(train_sentences))

tokenized_train_sentences = tokenizer.texts_to_sequences(train_sentences)
tokenized_test_sentences = tokenizer.texts_to_sequences(test_sentences)

train_padding = pad_sequences(tokenized_train_sentences, max_len)
test_padding = pad_sequences(tokenized_test_sentences, max_len)

### Create embeddings matrix
- Download embeddings with Magnitude libray
- Create an embedding_matrix dims: number_of_words x embeddings.dim with zero values
- Fill the embedding_matrix with the embeddings with .query() Magnitude's function

In [4]:
#max_len = 150
#https://github.com/plasticityai/magnitude
#!curl -s http://magnitude.plasticity.ai/glove+subword/glove.6B.300d.magnitude --output vectors.magnitude

#vecs_word2vec = Magnitude('http://magnitude.plasticity.ai/word2vec/heavy/GoogleNews-vectors-negative300.magnitude', stream=True, pad_to_length=max_len) 
vecs_glove = Magnitude('http://magnitude.plasticity.ai/glove+subword/glove.6B.300d.magnitude')
vecs_fasttext = Magnitude('http://magnitude.plasticity.ai/fasttext+subword/wiki-news-300d-1M.magnitude', pad_to_length=max_len)
#vecs_elmo = Magnitude('http://magnitude.plasticity.ai/elmo/medium/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.magnitude', stream=True, pad_to_length=max_len)

#vectors = Magnitude(vecs_fasttext, vecs_glove) # concatenate word2vec with glove

In [5]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, vecs_glove.dim))

from tqdm import tqdm_notebook as tqdm
for word, i in tqdm(word_index.items()):
    if i >= max_features:
        continue
    embedding_vector = vecs_glove.query(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.random.uniform(-0.25, 0.25, embed_size)

gc.collect()

### Model Helpers
Initialize the custome classes/functions that we'll need for our models

- RocAuc metric

In [6]:
# https://www.kaggle.com/yekenot/pooled-gru-fasttext

#Define a class for model evaluation
class RocAucEvaluation(Callback):
    def __init__(self, training_data=(),validation_data=()):
        super(Callback, self).__init__()
       
        self.X_tra, self.y_tra = training_data
        self.X_val, self.y_val = validation_data
        self.aucs_val = []
        self.aucs_tra = []
        
    def on_epoch_end(self, epoch, logs={}):                   
        y_pred_val = self.model.predict(self.X_val, verbose=0)
        score_val = roc_auc_score(self.y_val, y_pred_val)

        y_pred_tra = self.model.predict(self.X_tra, verbose=0)
        score_tra = roc_auc_score(self.y_tra, y_pred_tra)

        self.aucs_tra.append(score_tra)
        self.aucs_val.append(score_val)
        print("\n ROC-AUC - epoch: %d - score_tra: %.6f - score_val: %.6f \n" % (epoch+1, score_tra, score_val))

def recall(y_true, y_pred):    
    """
    Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision(y_true, y_pred):    
    """
    Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    Source
    ------
    https://github.com/fchollet/keras/issues/5400#issuecomment-314747992
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1(y_true, y_pred):
    
    """Calculate the F1 score."""
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r))


def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)), axis=1)

In [7]:
class Plots:
    def plot_history(history):
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        x = range(1, len(val_loss) + 1)

        plt.plot(x, loss, 'b', label='Training loss')
        plt.plot(x, val_loss, 'r', label='Validation loss')
        plt.title('Training and validation loss')
        plt.legend()

    def plot_roc_auc(train_roc, val_roc):
        x = range(1, len(val_roc) + 1)

        plt.plot(x, train_roc, 'b', label='Training RocAuc')
        plt.plot(x, val_roc, 'r', label='Validation RocAuc')
        plt.title('Training and validation RocAuc')
        plt.legend()

### Create models
- BaseLine models https://realpython.com/python-keras-text-classification/
- Single mode 98.18: https://github.com/ipcplusplus/toxic-comments-classification/blob/master/toxic_comment_analysis.ipynb
- Attention Display : https://github.com/conversationai/conversationai-models/blob/master/attention-tutorial/Attention_Model_Tutorial.ipynb
- Attention Models: https://github.com/thinline72/toxic/tree/master/skolbachev/toxic
- Many models: https://github.com/neptune-ml/open-solution-toxic-comments
- More modes alno: https://github.com/alno/kaggle-jigsaw-toxic-comment-classification-challenge

In [8]:
X_tra, X_val, y_tra, y_val = train_test_split(train_padding, y, train_size=0.90, random_state=233)
RocAuc = RocAucEvaluation(training_data=(X_tra, y_tra) ,validation_data=(X_val, y_val))

### Simple RNN

Text data have a sequence. Thus, the meaning of a word is dependant on the previous words. Thus, we will try to use RNN that uses the previous state of the sequence.

In [9]:
input_layer = Input(shape=(max_len, ))
X = Embedding(max_features, embed_size, weights=[embedding_matrix])(input_layer)
X = SimpleRNN(units=128, activation="relu")(X)
X = Dense(6, activation="sigmoid")(X)
model = Model(inputs=input_layer, outputs=X)

model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy', 'binary_crossentropy'])

saved_model = "weights_base.best.hdf5"

checkpoint = ModelCheckpoint(saved_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=2)
callbacks_list = [checkpoint, early, RocAuc]

batch_sz = 64
epoch = 20

model.fit(X_tra,
          y_tra,
          validation_data=(X_val, y_val),
          batch_size=batch_sz,
          epochs=epoch,
          callbacks=callbacks_list,
          shuffle=True,
          verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/20
143613/143613 [==============================] - 197s 1ms/step - loss: 0.3589 - acc: 0.9680 - binary_crossentropy: 0.3589 - val_loss: 0.5962 - val_acc: 0.9628 - val_binary_crossentropy: 0.5962

Epoch 00001: val_loss improved from inf to 0.59619, saving model to weights_base.best.hdf5

 ROC-AUC - epoch: 1 - score_tra: 0.509557 - score_val: 0.513197 

Epoch 2/20
143613/143613 [==============================] - 194s 1ms/step - loss: 0.5817 - acc: 0.9638 - binary_crossentropy: 0.5817 - val_loss: 0.5977 - val_acc: 0.9628 - val_binary_crossentropy: 0.5977

Epoch 00002: val_loss did not improve from 0.59619

 ROC-AUC - epoch: 2 - score_tra: 0.508895 - score_val: 0.511122 

Epoch 3/20
143613/143613 [==============================] - 195s 1ms/step - loss: 0.5821 - acc: 0.9638 - binary_crossentropy: 0.5821 - val_loss: 0.5977 - val_acc: 0.9628 - val_binary_crossentropy: 0.5977

Epoch 00003: val_loss did not improve from 0.59619

 ROC-A

### Bidirectional RNN
From the simple RNN we saw that using the information from the previous state of the sequence helps, hence we will try to use a biRNN in order to use information that is not only before that token but also after

In [10]:
input_layer = Input(shape=(max_len, ))
X = Embedding(max_features, embed_size, weights=[embedding_matrix])(input_layer)
X = Bidirectional(CuDNNGRU(128))(X)
X = Dense(6, activation="sigmoid")(X)
model = Model(inputs=input_layer, outputs=X)

model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy', 'binary_crossentropy'])

saved_model = "weights_base.best.hdf5"

checkpoint = ModelCheckpoint(saved_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=2)
callbacks_list = [checkpoint, early, RocAuc]

batch_sz = 128
epoch = 20

model.fit(X_tra,
          y_tra,
          validation_data=(X_val, y_val),
          batch_size=batch_sz,
          epochs=epoch,
          callbacks=callbacks_list,
          shuffle=True,
          verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/20
143613/143613 [==============================] - 45s 313us/step - loss: 0.0611 - acc: 0.9797 - binary_crossentropy: 0.0611 - val_loss: 0.0455 - val_acc: 0.9832 - val_binary_crossentropy: 0.0455

Epoch 00001: val_loss improved from inf to 0.04546, saving model to weights_base.best.hdf5

 ROC-AUC - epoch: 1 - score_tra: 0.987661 - score_val: 0.981955 

Epoch 2/20
143613/143613 [==============================] - 43s 296us/step - loss: 0.0358 - acc: 0.9859 - binary_crossentropy: 0.0358 - val_loss: 0.0442 - val_acc: 0.9832 - val_binary_crossentropy: 0.0442

Epoch 00002: val_loss improved from 0.04546 to 0.04424, saving model to weights_base.best.hdf5

 ROC-AUC - epoch: 2 - score_tra: 0.996138 - score_val: 0.985598 

Epoch 3/20
143613/143613 [==============================] - 43s 297us/step - loss: 0.0253 - acc: 0.9900 - binary_crossentropy: 0.0253 - val_loss: 0.0509 - val_acc: 0.9821 - val_binary_crossentropy: 0.0509

Epoch 00003

### BiGRU with ebedding projection layer

In [11]:
input_layer = Input(shape=(max_len, ))
X = Embedding(max_features, embed_size, weights=[embedding_matrix])(input_layer)
X = Dense(units=max_len, activation='relu')(X)
X = BatchNormalization()(X)
X = Bidirectional(CuDNNGRU(128))(X)
X = Dense(6, activation="sigmoid")(X)
model = Model(inputs=input_layer, outputs=X)

model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy', 'binary_crossentropy'])

saved_model = "weights_base.best.hdf5"

checkpoint = ModelCheckpoint(saved_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks_list = [checkpoint, early, RocAuc]

batch_sz = 128
epoch = 20

model.fit(X_tra,
          y_tra,
          validation_data=(X_val, y_val),
          batch_size=batch_sz,
          epochs=epoch,
          callbacks=callbacks_list,
          shuffle=True,
          verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/20
143613/143613 [==============================] - 47s 324us/step - loss: 0.0533 - acc: 0.9805 - binary_crossentropy: 0.0533 - val_loss: 0.0429 - val_acc: 0.9838 - val_binary_crossentropy: 0.0429

Epoch 00001: val_loss improved from inf to 0.04293, saving model to weights_base.best.hdf5

 ROC-AUC - epoch: 1 - score_tra: 0.992879 - score_val: 0.986464 

Epoch 2/20
143613/143613 [==============================] - 44s 305us/step - loss: 0.0317 - acc: 0.9875 - binary_crossentropy: 0.0317 - val_loss: 0.0462 - val_acc: 0.9828 - val_binary_crossentropy: 0.0462

Epoch 00002: val_loss did not improve from 0.04293

 ROC-AUC - epoch: 2 - score_tra: 0.997677 - score_val: 0.984982 

Epoch 3/20
143613/143613 [==============================] - 44s 305us/step - loss: 0.0207 - acc: 0.9920 - binary_crossentropy: 0.0207 - val_loss: 0.0557 - val_acc: 0.9813 - val_binary_crossentropy: 0.0557

Epoch 00003: val_loss did not improve from 0.04293

 RO

### BiGRU with MLP on top and embeddings projection layer

In [12]:
input_layer = Input(shape=(max_len, ))
X = Embedding(max_features, embed_size, weights=[embedding_matrix])(input_layer)
# Embedding projection Layer before the RNN
X = Dense(units=max_len, activation='relu')(X)
# X = Dropout(0.2)(X)
X = BatchNormalization()(X)
X = Bidirectional(CuDNNGRU(128))(X)
# MLP on top of BiGRU
X = Dense(256, activation='relu' )(X)
# X = Dense(100, activation='relu' )(X)
X = Dense(6, activation="sigmoid")(X)
model = Model(inputs=input_layer, outputs=X)

model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy', 'binary_crossentropy'])

saved_model = "weights_base.best.hdf5"

checkpoint = ModelCheckpoint(saved_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks_list = [checkpoint, early, RocAuc]

batch_sz = 128
epoch = 20

model.fit(X_tra,
          y_tra,
          validation_data=(X_val, y_val),
          batch_size=batch_sz,
          epochs=epoch,
          callbacks=callbacks_list,
          shuffle=True,
          verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/20
143613/143613 [==============================] - 47s 326us/step - loss: 0.0517 - acc: 0.9811 - binary_crossentropy: 0.0517 - val_loss: 0.0443 - val_acc: 0.9831 - val_binary_crossentropy: 0.0443

Epoch 00001: val_loss improved from inf to 0.04429, saving model to weights_base.best.hdf5

 ROC-AUC - epoch: 1 - score_tra: 0.993332 - score_val: 0.986613 

Epoch 2/20
143613/143613 [==============================] - 44s 306us/step - loss: 0.0322 - acc: 0.9870 - binary_crossentropy: 0.0322 - val_loss: 0.0462 - val_acc: 0.9832 - val_binary_crossentropy: 0.0462

Epoch 00002: val_loss did not improve from 0.04429

 ROC-AUC - epoch: 2 - score_tra: 0.997458 - score_val: 0.985636 

Epoch 3/20
143613/143613 [==============================] - 44s 305us/step - loss: 0.0216 - acc: 0.9915 - binary_crossentropy: 0.0216 - val_loss: 0.0585 - val_acc: 0.9808 - val_binary_crossentropy: 0.0585

Epoch 00003: val_loss did not improve from 0.04429

 RO

In [13]:
input_layer = Input(shape=(max_len, ))
X = Embedding(max_features, embed_size, weights=[embedding_matrix])(input_layer)
# Embedding projection Layer before the RNN
X = Dense(units=max_len, activation='relu')(X)
# X = Dropout(0.2)(X)
X = BatchNormalization()(X)
x_state, x_fwd, x_bwd = Bidirectional(CuDNNGRU(128, return_sequences=True, return_state=True))(X)
X = concatenate([x_fwd, x_bwd])
# MLP on top of BiGRU
# X = Reshape((2 * max_len,128, 1))(X)
X = Dense(units=256, activation='relu')(X)
X = Dense(6, activation="sigmoid")(X)
model = Model(inputs=input_layer, outputs=X)

model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy', 'binary_crossentropy'])

saved_model = "weights_base.best.hdf5"

checkpoint = ModelCheckpoint(saved_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks_list = [checkpoint, early, RocAuc]

batch_sz = 128
epoch = 20

model.fit(X_tra,
          y_tra,
          validation_data=(X_val, y_val),
          batch_size=batch_sz,
          epochs=epoch,
          callbacks=callbacks_list,
          shuffle=True,
          verbose=1)
model.summary()

Train on 143613 samples, validate on 15958 samples
Epoch 1/20
143613/143613 [==============================] - 47s 329us/step - loss: 0.0528 - acc: 0.9807 - binary_crossentropy: 0.0528 - val_loss: 0.0441 - val_acc: 0.9833 - val_binary_crossentropy: 0.0441

Epoch 00001: val_loss improved from inf to 0.04409, saving model to weights_base.best.hdf5

 ROC-AUC - epoch: 1 - score_tra: 0.992439 - score_val: 0.986435 

Epoch 2/20
143613/143613 [==============================] - 44s 307us/step - loss: 0.0323 - acc: 0.9871 - binary_crossentropy: 0.0323 - val_loss: 0.0463 - val_acc: 0.9828 - val_binary_crossentropy: 0.0463

Epoch 00002: val_loss did not improve from 0.04409

 ROC-AUC - epoch: 2 - score_tra: 0.997546 - score_val: 0.985154 

Epoch 3/20
143613/143613 [==============================] - 44s 307us/step - loss: 0.0215 - acc: 0.9915 - binary_crossentropy: 0.0215 - val_loss: 0.0556 - val_acc: 0.9810 - val_binary_crossentropy: 0.0556

Epoch 00003: val_loss did not improve from 0.04409

 RO

## Stacked BiGRU

In [14]:
target_shape = 6
lr=0.0003
rnn_dropout=None
rnn_layers=[128, 64]
mlp_layers=[70]
mlp_dropout=0.1
text_emb_dropout=0.0
text_emb_size=300

model = Sequential()
model.add(InputLayer(name='comment_text', input_shape=[max_len]))
model.add(Embedding(max_features, text_emb_size, weights=[embedding_matrix], trainable=False))
model.add(Dropout(text_emb_dropout))

for layer_size in rnn_layers:
    #Fast LSTM implementation backed by CuDNN. Can only be run on GPU, with the TensorFlow backend.
    model.add(Bidirectional(CuDNNLSTM(layer_size, return_sequences=True)))
    if rnn_dropout is not None:
        model.add(SpatialDropout1D(rnn_dropout))

model.add(GlobalMaxPool1D())
for layer_size in mlp_layers:
    model.add(Dense(layer_size, activation="relu"))
    model.add(Dropout(mlp_dropout))
model.add(Dense(6, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000015))

saved_model = "weights_base.best.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
callbacks_list = [checkpoint, early, RocAuc]

model.fit(x=X_tra,
          y=y_tra,
          validation_data=(X_val, y_val),
          batch_size=128,
          epochs=80,         
          callbacks=callbacks_list, verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/80
143613/143613 [==============================] - 59s 408us/step - loss: 0.1044 - val_loss: 0.0664

Epoch 00001: val_loss improved from inf to 0.06639, saving model to weights_base.best.hdf5

 ROC-AUC - epoch: 1 - score_tra: 0.960413 - score_val: 0.958003 

Epoch 2/80
143613/143613 [==============================] - 57s 394us/step - loss: 0.0609 - val_loss: 0.0574

Epoch 00002: val_loss improved from 0.06639 to 0.05745, saving model to weights_base.best.hdf5

 ROC-AUC - epoch: 2 - score_tra: 0.968417 - score_val: 0.966508 

Epoch 3/80
143613/143613 [==============================] - 57s 394us/step - loss: 0.0553 - val_loss: 0.0563

Epoch 00003: val_loss improved from 0.05745 to 0.05634, saving model to weights_base.best.hdf5

 ROC-AUC - epoch: 3 - score_tra: 0.971904 - score_val: 0.969387 

Epoch 4/80
143613/143613 [==============================] - 57s 394us/step - loss: 0.0535 - val_loss: 0.0545

Epoch 00004: val_loss impro

In [15]:
K.clear_session()
del model
gc.collect()

113

In [16]:
# model.summary() # Print a description of the model.
# Plots.plot_roc_auc(RocAuc.aucs_tra, RocAuc.aucs_val)
Plots.plot_history(model.history)

NameError: name 'model' is not defined

In [17]:
test_values = model.predict([test_padding], batch_size=1024, verbose=1)
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission[classes] = test_values
sample_submission.to_csv('/submission.csv', index=False)

NameError: name 'model' is not defined

In [18]:
from IPython.display import HTML
import pandas as pd
import numpy as np

sample_submission.to_csv('submission.csv', index=False)

def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission.csv')

NameError: name 'sample_submission' is not defined

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

NameError: name 'model' is not defined

In [20]:
!pip install ann_visualizer
!pip install graphviz
!pip install h5py
from ann_visualizer.visualize import ann_viz
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
ann_viz(model, title="Artificial Neural network - Model Visualization")

  Stored in directory: /tmp/.cache/pip/wheels/b6/b4/4e/d92f50c9c4f004cf315a0e0fcd455486bd799c50fe80cf1f5d
Successfully built ann-visualizer


FileNotFoundError: [Errno 2] No such file or directory: 'xdg-open': 'xdg-open'

In [21]:
from keras.models import load_model
model = load_model('model.h5')

OSError: Unable to open file (unable to open file: name = 'model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)